# Make Images for Gifs

Please note: this script requires specific manual inputs and is not intended to be general.

This code reads in fits files, and creates moment map images in RA/DEC. It draws contours at 3,5,10,20,50 sigma. The purpose of this script was originally to make images to be turned into gifs, as such all images are roughly on the same scale and it requires a lot of manual inputs. A different version of this code is more general and more automated. Other version should be "Contunuum_images".

In [6]:
#imports needed to make images

%matplotlib inline
import numpy as np
from spectral_cube import SpectralCube
from astropy import units as u
from matplotlib import pyplot as plt
import warnings 
from astropy.coordinates import SkyCoord
from astroquery.splatalogue import Splatalogue
import aplpy
from astropy.wcs.utils import proj_plane_pixel_scales
from astropy.wcs import WCS
from astropy.io import fits
from scipy.signal import find_peaks

warnings.filterwarnings('ignore') # turns of annoying red warning boxes


## Upload files to make images

I made three gifs with these images: a gif of each ALMA band at its natural beam size and cell size, a gif of each of the band 7 data at its natural beam size and cell size, and a gif of band 7 observations smoothed and regridded to the same beam size and same cell size.

In [7]:
# for emission across all bands gif
b7 ='/Users/hannahshoemaker/VICO_dust/HH212/HH212data/band7_hannah.fits'
b9 ='/Users/hannahshoemaker/VICO_dust/HH212/HH212data/band9_hannah.fits'
b3 ='/Users/hannahshoemaker/VICO_dust/HH212/HH212data/band3_hannah.fits'
b6 ='/Users/hannahshoemaker/VICO_dust/HH212/HH212data/band6_hannah.fits'

# for emission across all band 7 observations gif

nat2012 = '/Users/hannahshoemaker/VICO_dust/HH212/HH212data/2012cont.fits'
nat2015 = '/Users/hannahshoemaker/VICO_dust/HH212/HH212data/2015.fits'
nat2016x259 = '/Users/hannahshoemaker/VICO_dust/HH212/HH212data/2016_x259.fits'
nat2016x25b = '/Users/hannahshoemaker/VICO_dust/HH212/HH212data/2016_x25b.fits'
nat2017 ='/Users/hannahshoemaker/VICO_dust/HH212/HH212data/2017.fits'

# for emission across all band 7 observations, smooth & regrid gif

smooth2012 = '/Users/hannahshoemaker/VICO_dust/HH212/HH212data/2012.smooth.fits'
smooth2015 = '/Users/hannahshoemaker/VICO_dust/HH212/HH212data/2015.smooth.fits'
smooth2016x259 = '/Users/hannahshoemaker/VICO_dust/HH212/HH212data/2016_x259_2.smooth.regrid.fits'
smooth2016x25b =  '/Users/hannahshoemaker/VICO_dust/HH212/HH212data/2016_x25b.regrid.fits'
smooth2017 = '/Users/hannahshoemaker/VICO_dust/HH212/HH212data/2017.smooth.regrid.fits'


In [9]:
def Read_Cube(file_name):
    '''
    Read_Cube takes the file name, uses the package SpectralCube to read it, takes the axes in Hz and returns the cube.
    It is necessary to convert the axes into Hz rather than velocity.
    '''
    cube = (SpectralCube.read(file_name))
    cube.spectral_axis
    cube2 = cube.with_spectral_unit(u.km/u.s , velocity_convention = 'radio', rest_value = cube.header['RESTFRQ'] *u.Hz)
    cube2.spectral_axis
    return cube2

def Draw_Box(cube,boxsize):
    '''
    Draw_Box takes the cube returned from Read_Cube and cuts the cube down into a smaller "cutcube". Returns cutcube.
    
    "boxsize" is set specific to each file in the case, since they need to be at roughly the same resolution.
    '''
    boxsize= boxsize
    drawbox_x = [int(cube.shape[2]/2-boxsize),int(cube.shape[2]/2+boxsize)]
    drawbox_y = [int(cube.shape[1]/2-boxsize),int(cube.shape[1]/2+boxsize)]
    cutcube = cube.subcube(xlo = drawbox_x[0],xhi = drawbox_x[1],ylo = drawbox_y[0],yhi = drawbox_y[1])
    return cutcube


def Contours(cube,cutcube,rms,boxsize,figname,additionaltxt):
    '''
    Contours takes the cube, cutcube, rms, boxside, figname and additional text and creates an image.
    1. Takes the cutcube and uses it as the background of the image
    2. Draws the beam in the bottom left corner of the image and sets axis labels
    draws contours based on rms
    3. Adds additional labels and text, depending onbox size
    4. saves figure as a pdf based on parameter 'figname'
    
    '''
    
    fig = aplpy.FITSFigure(cutcube[0,:,:].hdu, dimensions=[0, 1])
    fig.add_beam(major=cube.header['bmaj'],minor=cube.header['bmin'],angle=cube.header['bpa'],fill=True,color='magenta')
    fig.axis_labels.show()
    fig.axis_labels.set_xtext("Right Ascension (ICRS)") 
    fig.axis_labels.set_ytext("Declination (ICRS)") 
    plt.imshow(cutcube[0,:,:].hdu.data, cmap = 'inferno') 
         
    cbar = plt.colorbar(shrink=0.9)
    #cbar.ax.set_ylabel('Moment 0 Map Intensity')

    levs_rms = np.arange(0,1000)*rms.value
        
    print('\u03C3 is', rms.value,'Jy/beam')
    
    fig.show_contour(cutcube[0,:,:].hdu.data,levels = levs_rms[3:4],colors='white',linestyles= 'dashed')
    fig.show_contour(cutcube[0,:,:].hdu.data,levels = levs_rms[5:6], colors='white')
    fig.show_contour(cutcube[0,:,:].hdu.data,levels = levs_rms[10:11], colors='white')
    fig.show_contour(cutcube[0,:,:].hdu.data,levels = levs_rms[20:21], colors='white')
    fig.show_contour(cutcube[0,:,:].hdu.data,levels = levs_rms[50:51], colors='white')
        
    sigma_text = '\u03C3 ='+ str(rms.value) + ' Jy/beam'
    font = {'family': 'veranda','color':  'white','weight': 'bold','size': 16}
        
    if boxsize == 130:
        plt.text(20, 230, sigma_text, fontdict=font)
        plt.text(20,215, additionaltxt,fontdict=font)
    elif boxsize ==50:
        plt.text(10,87, sigma_text, fontdict=font)
        plt.text(10,82, additionaltxt,fontdict=font)
    elif boxsize ==30:
        plt.text(5,53, sigma_text, fontdict=font)
        plt.text(5,50, additionaltxt,fontdict=font)
    elif boxsize ==80:
        plt.text(10,145, sigma_text, fontdict=font)
        plt.text(5,50, additionaltxt,fontdict=font)
    else:
        print("I DO NOT KNOW WHERE TO PUT YOUR ADDITIONAL TEXT")
            
     
    fig.savefig(figname+'.png')
        

def Main(file_name,rms,boxsize,figname,additionaltxt):
    '''
    Main takes file name, rms, box size, fig name and additional text and uses functions defined above to make image.
    
    It should be noted all the parameters taken by Main are incredibly observation-specific and are not general in any way.

    '''
    cube = Read_Cube(file_name)
    cutcube = Draw_Box(cube,boxsize)
    rms = rms *u.Jy
    Contours(cube,cutcube,rms,boxsize,figname,additionaltxt)


In [ ]:
#Main(b7, 0.0000522*u.Jy,130,'b7','ALMA BAND 7')

In [ ]:
#Main(b9,0.00266 *u.Jy,50,'b9','ALMA BAND 9')

In [ ]:
#Main(b3,0.000008576 *u.Jy,130,'b3','ALMA BAND 3')

In [ ]:
#Main(b6,0.00002658 *u.Jy,130,'b6','ALMA BAND 6')

In [ ]:
#Main(nat2012,0.004 *u.Jy,30,'nat2012','')

In [ ]:
#Main(nat2015,0.0001662 *u.Jy,130,'nat2015','')

In [ ]:
#Main(nat2016x259,0.002232 *u.Jy,50,'nat2016x259','')

In [ ]:
#Main(nat2016x25b,0.00367 *u.Jy,30,'nat2016x25b','')

In [ ]:
#Main(nat2017,0.000064 *u.Jy,80,'nat2017','')

In [ ]:
#Main(smooth2012,0.00272 *u.Jy,80,'smooth2012','')

In [ ]:
#Main(smooth2015,0.00288 *u.Jy,80,'smooth2015','')

In [ ]:
#Main(smooth2016x25b,0.0006 *u.Jy,80,'smooth2016x25b','')

In [ ]:
#Main(smooth2017,0.000436 *u.Jy,80,'smooth2017','')

In [ ]:
#Main(smooth2016x259, 0.002044* u.Jy ,80, 'smooth2016x259', '')
